In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

In [3]:
from langchain_openai import ChatOpenAI
llm_llama70b = ChatOpenAI(
    base_url="https://api.together.xyz/v1", #https://api.together.xyz
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    temperature=0,)

In [4]:
llm_transformer_llama70b = LLMGraphTransformer(llm=llm_llama70b)

In [5]:
def showGrath():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

In [6]:
showGrath()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [15]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [7]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = prompt | llm_llama70b.with_structured_output(Entities)

In [8]:
entity_chain.invoke({"question":"¿Quién es Úrsula?"})

Entities(names=['Úrsula'])

In [9]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [10]:
print(graph_retriever("¿quién es Úrsula?"))

Úrsula - INFLUENCED -> Población
Úrsula - INFLUENCED -> Miró
Úrsula - INTERACTED_WITH -> José Arcadio
Úrsula - SPOUSE -> José Arcadio Buendía
Úrsula - PROTECTED -> Aureliano
Úrsula - PROTECTED -> José Arcadio
Úrsula - VISITED -> La Casa
Úrsula - SETTLEMENT -> Macondo
Úrsula - SPEAKS_TO -> José Arcadio Buendía
Úrsula - SPOKE_TO -> El Marido
Úrsula - MOTHER_OF -> José Arcadio
Úrsula - SEES -> Prudencio Aguilar
Úrsula - HELPS -> Prudencio Aguilar
Úrsula - WIFE -> José Arcadio Buendía
Úrsula - USED -> Los Naipes
Úrsula - ESPOSOS -> José Arcadio Buendía
Úrsula - FAMILY -> Amaranta
Úrsula - SEARCH -> José Arcadio
Úrsula - MADRE_E_HIJO -> Hijo De Úrsula
Úrsula Iguarán - DISUADIRLO -> José Arcadio Buendía
José Arcadio Buendía - MARRIED_TO -> Úrsula
Habiendo - RELATIONSHIP -> Úrsula
José Arcadio Buendía - SPOUSE -> Úrsula
José Arcadio Buendía - TALKED_TO -> Úrsula
José Arcadio Buendía - SPEAKS_TO -> Úrsula
José Arcadio Buendía - HUSBAND -> Úrsula
Miró - SPOKE_TO -> Úrsula
Aureliano - CHILD -> Ú

In [11]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm_llama70b
    | StrOutputParser()
)

In [16]:
chain.invoke(input="¿Quién es Úrsula?")

'Úrsula es la esposa de José Arcadio Buendía y la madre de José Arcadio y Aureliano.'

In [16]:
chain.invoke(input="¿Dónde vivió José Arcadio Buendía?")

'José Arcadio Buendía vivió en Macondo.'

In [17]:
chain.invoke(input="¿Dónde vivió el esposo de Úrsula?")

'No se menciona explícitamente dónde vivió el esposo de Úrsula, pero se puede inferir que vivió en la aldea donde se desarrolla la historia, ya que se menciona que la gente de la aldea se apretujaba en el laboratorio de José Arcadio Buendía para celebrar el prodigio de la alquimia.'

In [18]:
chain.invoke(input="¿Dónde vive el padre de Aureliano Buendía?")

'No se menciona explícitamente dónde vive el padre de Aureliano Buendía. Sin embargo, se menciona que Aureliano Buendía vive en Macondo.'

In [20]:
chain.invoke(input="¿Cuáles son las ciudades visitadas por Melquiades")

'No se mencionan ciudades visitadas por Melquíades en el texto proporcionado.'

In [21]:
chain.invoke(input="¿Quién tenía una fuerza descomunal?")

'El gitano no, sino que era José Arcadio Buendía quien tenía una fuerza descomunal, que le permitía derribar un caballo agarrándolo por las orejas.'

In [28]:
chain.invoke(input="¿a qué personaje el escorbuto le había arrancado los dientes?")

'A Melquíades.'

In [36]:
chain.invoke(input="¿cuántos doblones de oro tenía Úrsula?")

'Treinta doblones de oro.'

In [48]:
chain.invoke(input="¿Que hizo José Arcadio Buendía con el oro, los doblones, de su esposa?")

'José Arcadio Buendía echó los doblones de su esposa en una cazuela, los fundió con otros materiales y los puso a hervir en un caldero de aceite de ricino, pero el resultado fue un chicharrón carbonizado que no pudo ser desprendido del fondo del caldero.'

In [49]:
chain.invoke(input="¿Cómo era la casa de Úrsula y José Arcadio Buendía?")

'La casa de Úrsula y José Arcadio Buendía era la mejor de la aldea. Tenía una salita amplia y bien iluminada, un comedor en forma de terraza con flores de colores alegres, dos dormitorios, un patio con un castaño gigantesco, un huerto bien plantado y un corral donde vivían en comunidad pacífica los chivos, los cerdos y las gallinas.'